In [1]:
# Importing the dataset

In [2]:
import pandas as pd
Train_dat=pd.read_csv('D:/ML_Projects/Fake_News_Classifier/train.csv')

In [3]:
Train_dat.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
#Dropping Nan Values
Train_dat=Train_dat.dropna()

In [5]:
# Getting X and Y
X=Train_dat.drop('label', axis=1)
Y=Train_dat['label']

In [6]:
print(X.shape)
print(Y.shape)

(18285, 4)
(18285,)


In [7]:
# Importing the Model libraries

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [9]:
# One Hot Representation

In [10]:
voc_size=5000

In [11]:
message=X.copy()
message.reset_index(inplace=True)

In [12]:
import nltk
import re
from nltk.corpus import stopwords

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\ROHIT
[nltk_data]     MANDAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0, len(message)):
    review=re.sub('[^A-Za-z]', ' ', message['title'][i])
    review=review.lower()
    review=review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [15]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 

print(len(onehot_repr))

18285


In [16]:
# Embedding Representation (Padding)

In [17]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1202 3706 1655]
 [   0    0    0 ... 2150  886 2907]
 [   0    0    0 ... 2552 1560 2768]
 ...
 [   0    0    0 ...  322  606 2474]
 [   0    0    0 ... 4285 3690 1301]
 [   0    0    0 ...  971 4964 4526]]


In [18]:
embedded_docs[5]

array([   0,    0,    0,    0,    0,   84, 3083, 3411,   92, 1290,  180,
       3898,  501, 3782, 1976, 1812, 4221, 3787, 3631, 2907])

In [19]:
# Creating the Model

In [20]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model1.summary

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x00000252885DC040>>

In [21]:
import numpy as np
X_final=np.array(embedded_docs)
Y_final=np.array(Y)

In [22]:
# Splitting the dataset

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test=train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

In [24]:
# Model Training

In [25]:
model1.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=30, batch_size=64)

Epoch 1/30
192/192 [==============================] - 10s 37ms/step - loss: 0.4491 - accuracy: 0.7518 - val_loss: 0.2101 - val_accuracy: 0.9170
Epoch 2/30
192/192 [==============================] - 5s 24ms/step - loss: 0.1295 - accuracy: 0.9521 - val_loss: 0.2303 - val_accuracy: 0.9160
Epoch 3/30
192/192 [==============================] - 5s 24ms/step - loss: 0.0881 - accuracy: 0.9702 - val_loss: 0.2346 - val_accuracy: 0.9089
Epoch 4/30
192/192 [==============================] - 5s 25ms/step - loss: 0.0721 - accuracy: 0.9769 - val_loss: 0.3070 - val_accuracy: 0.9118
Epoch 5/30
192/192 [==============================] - 5s 26ms/step - loss: 0.0491 - accuracy: 0.9862 - val_loss: 0.3430 - val_accuracy: 0.9049
Epoch 6/30
192/192 [==============================] - 5s 25ms/step - loss: 0.0375 - accuracy: 0.9887 - val_loss: 0.3811 - val_accuracy: 0.9084
Epoch 7/30
192/192 [==============================] - 5s 26ms/step - loss: 0.0295 - accuracy: 0.9920 - val_loss: 0.4388 - val_accuracy: 0.905

In [26]:
# Accuracy Score

In [27]:
from sklearn.metrics import accuracy_score
Y_pred1=model1.predict_classes(X_test)
accuracy_score(Y_test,Y_pred1)

C:\Users\ROHIT MANDAL\AppData\Local\Programs\Python\Python38\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


0.8995857497928749